In [1]:
import pandas as pd

In [2]:
# Reading the data

df = pd.read_csv('/Users/adityanittala/Documents/Carnegie Mellon University/Semester - 3/Interactive Data Science/Assignment 3/abnormal_distribution/MigrationPatternsData/od.csv')
df.info(verbose=True, show_counts=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13727025 entries, 0 to 13727024
Data columns (total 12 columns):
 #   Column        Non-Null Count     Dtype  
---  ------        --------------     -----  
 0   o_cz          13727025 non-null  int64  
 1   o_cz_name     13727025 non-null  object 
 2   o_state_name  13727025 non-null  object 
 3   d_cz          13727025 non-null  int64  
 4   d_cz_name     13727025 non-null  object 
 5   d_state_name  13727025 non-null  object 
 6   n             13727025 non-null  int64  
 7   n_tot_o       13727025 non-null  int64  
 8   n_tot_d       13727025 non-null  int64  
 9   pool          13727025 non-null  object 
 10  pr_d_o        13139412 non-null  float64
 11  pr_o_d        13125333 non-null  float64
dtypes: float64(2), int64(5), object(5)
memory usage: 1.2+ GB


In [3]:
df.head(10)

,o_cz,o_cz_name,o_state_name,d_cz,d_cz_name,d_state_name,n,n_tot_o,n_tot_d,pool,pr_d_o,pr_o_d
0,100,Johnson City,Tennessee,10000,Panama City,Florida,0,36,44,AsianQ1,0.0,0.0
1,100,Johnson City,Tennessee,31303,Kerrville,Texas,0,36,2,AsianQ1,0.0,0.0
2,100,Johnson City,Tennessee,10600,Jasper,Alabama,0,36,6,AsianQ1,0.0,0.0
3,100,Johnson City,Tennessee,2600,Roanoke Rapids,North Carolina,0,36,8,AsianQ1,0.0,0.0
4,100,Johnson City,Tennessee,11402,Marinette,Michigan,0,36,5,AsianQ1,0.0,0.0
5,100,Johnson City,Tennessee,35401,Flagstaff,Arizona,0,36,22,AsianQ1,0.0,0.0
6,100,Johnson City,Tennessee,26704,Grand Forks,North Dakota,0,36,5,AsianQ1,0.0,0.0
7,100,Johnson City,Tennessee,26102,Marshall,Missouri,0,36,1,AsianQ1,0.0,0.0
8,100,Johnson City,Tennessee,25300,Union City,Tennessee,0,36,9,AsianQ1,0.0,0.0
9,100,Johnson City,Tennessee,32306,Eagle Pass,Texas,0,36,1,AsianQ1,0.0,0.0


In [24]:
grouped_df = df.groupby(['o_state_name', 'd_state_name', 'pool'])['n']\
                .agg('sum')\
                .reset_index()
grouped_df

,o_state_name,d_state_name,pool,n
0,Alabama,Alabama,AsianQ1,296
1,Alabama,Alabama,AsianQ2,272
2,Alabama,Alabama,AsianQ3,304
3,Alabama,Alabama,AsianQ4,271
4,Alabama,Alabama,AsianQ5,310
...,...,...,...,...
65020,Wyoming,Wyoming,WhiteQ1,3710
65021,Wyoming,Wyoming,WhiteQ2,5298
65022,Wyoming,Wyoming,WhiteQ3,8662
65023,Wyoming,Wyoming,WhiteQ4,10717


In [25]:
# Splitting the pool in to state and quantile

splits = grouped_df['pool'].str.split('Q', expand=True)
grouped_df['race'] = splits[0]
grouped_df['quintile'] = splits[1]
grouped_df.drop(['pool'], axis=1, inplace=True)
grouped_df

,o_state_name,d_state_name,n,race,quintile
0,Alabama,Alabama,296,Asian,1
1,Alabama,Alabama,272,Asian,2
2,Alabama,Alabama,304,Asian,3
3,Alabama,Alabama,271,Asian,4
4,Alabama,Alabama,310,Asian,5
...,...,...,...,...,...
65020,Wyoming,Wyoming,3710,White,1
65021,Wyoming,Wyoming,5298,White,2
65022,Wyoming,Wyoming,8662,White,3
65023,Wyoming,Wyoming,10717,White,4


In [28]:
# Adding unique identifier for source and destination states

## Source state
state_id_dict = {}

for idx, state in enumerate(grouped_df['o_state_name'].unique()):
    state_id_dict[state] = idx + 1

def assign_state_id(state):
    return state_id_dict[state]

grouped_df['o_state_id'] = grouped_df.apply(lambda x: assign_state_id(x['o_state_name']), axis=1)
grouped_df['d_state_id'] = grouped_df.apply(lambda x: assign_state_id(x['d_state_name']), axis=1)

col_order = ['o_state_id', 'o_state_name', 'd_state_id', 'd_state_name',
             'race', 'quintile', 'n']
grouped_df = grouped_df.reindex(columns=col_order)

grouped_df

,o_state_id,o_state_name,d_state_id,d_state_name,race,quintile,n
0,1,Alabama,1,Alabama,Asian,1,296
1,1,Alabama,1,Alabama,Asian,2,272
2,1,Alabama,1,Alabama,Asian,3,304
3,1,Alabama,1,Alabama,Asian,4,271
4,1,Alabama,1,Alabama,Asian,5,310
...,...,...,...,...,...,...,...
65020,51,Wyoming,51,Wyoming,White,1,3710
65021,51,Wyoming,51,Wyoming,White,2,5298
65022,51,Wyoming,51,Wyoming,White,3,8662
65023,51,Wyoming,51,Wyoming,White,4,10717


In [30]:
grouped_df.to_csv('state_to_state_migration.csv')